In [1]:
import Ipynb_importer

In [2]:
%%capture #忽略输出
import s07_训练数据生成 as datagen

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
import numpy as np
edges, nodes = datagen.load_data("edgeOSM.txt","nodeOSM.txt")
G = datagen.create_networkx_graph(edges, nodes)

In [5]:
TRAJ_MAX_LENGTH = 126  # 输入最长128 要给前后缀留下位置

In [6]:
print("边总量：",len(edges))

边总量： 4504


In [7]:
path = datagen.generate_path(G)  # 路径
traj, coord, dis = datagen.generate_traj(G, path) # 详细轨迹
sampled_traj, sampled_coord = datagen.sample_lists(traj, coord, min(TRAJ_MAX_LENGTH,int(len(traj)*0.6))) #采样
noisy_coord = [datagen.add_gaussian_noise(c, np.random.randint(10, 25)) for c in sampled_coord]  #加噪

print('距离：',dis)
print('坐标点个数：',len(traj))
print("采样后个数：",len(sampled_traj))

import json
print(json.dumps(traj))
print(json.dumps(sampled_traj))
print(json.dumps(sampled_coord))
print(json.dumps(noisy_coord))

距离： 904.8232147463399
坐标点个数： 41
采样后个数： 24
[3953, 3953, 3955, 3134, 1767, 3132, 3957, 3130, 3959, 2354, 1771, 3961, 3961, 1678, 3128, 3963, 3965, 3124, 1617, 3126, 3967, 3120, 1619, 3118, 3969, 1621, 3433, 3116, 1634, 1642, 3115, 1647, 3114, 1650, 1656, 1625, 1623, 1671, 1648, 2356, 2357]
[3134, 1767, 3132, 3957, 1771, 3961, 3128, 3963, 3965, 3124, 3126, 3120, 3118, 3969, 1621, 3433, 3116, 1634, 1642, 3115, 3114, 1656, 1671, 2357]
[[40.5637929, -3.4932821], [40.563762, -3.4933658], [40.5637435, -3.4934599], [40.5637395, -3.4935679], [40.563933, -3.4943782], [40.5642965, -3.49532895], [40.5649064, -3.4969468], [40.5649295, -3.4970793], [40.5649474, -3.4972168], [40.5649496, -3.4973691], [40.564897, -3.4980827], [40.5649122, -3.4983829], [40.5650484, -3.499265], [40.5651363, -3.4995969], [40.5653204, -3.5002325], [40.5653797, -3.5003264], [40.5654663, -3.5005335], [40.5655311, -3.5006659], [40.565656, -3.500824], [40.5657922, -3.5009859], [40.565879, -3.5011519], [40.5657948, -3.501223], 

In [8]:
def generate_noisy_traj(G):
    """
    This function takes a networkx graph 'G' as input and generates a noisy trajectory.
    """
    path = datagen.generate_path(G)
    traj, coord, dis = datagen.generate_traj(G, path)
    sampled_traj, sampled_coord = datagen.sample_lists(traj, coord, min(TRAJ_MAX_LENGTH,int(len(traj)*0.6)))
    noisy_coord = [datagen.add_gaussian_noise(c, np.random.randint(10, 25)) for c in sampled_coord]
    return sampled_traj, noisy_coord

In [9]:
traj,coords = generate_noisy_traj(G)

print('SRC -> ',json.dumps(coords))
print('TGT -> ',json.dumps(traj))

SRC ->  [[40.5524584201445, -3.495878735791361], [40.55252185259225, -3.496087745054628], [40.552938008049686, -3.4961936433709764], [40.552998967116196, -3.4958868029113965], [40.55265195373048, -3.4960654820215353], [40.55261424440494, -3.496138156253101]]
TGT ->  [1174, 2045, 1175, 1177, 1181, 264]


# 转换为词典

In [10]:
import utm

# 经纬度坐标
lon, lat = -3, 40.5

# 将经纬度坐标转换为UTM坐标
x, y, zone_num, zone_letter = utm.from_latlon(lat, lon)

# 输出UTM坐标
print("UTM X 坐标为：", x)
print("UTM Y 坐标为：", y)
print("所在UTM投影区域为：", zone_num, zone_letter)

UTM X 坐标为： 500000.0
UTM Y 坐标为： 4483254.73193229
所在UTM投影区域为： 30 T


In [11]:
import pandas as pd
# 定义转换函数
def convert_to_utm(row):
    lon, lat = row['lon'], row['lat']
    x, y, zone_num, zone_letter = utm.from_latlon(lat, lon)
    return pd.Series({'zone_num': zone_num, 'zone_letter': zone_letter,'x': x, 'y': y})

# 对每行数据进行转换
nodes[[ 'zone_num', 'zone_letter','x', 'y']] = nodes.apply(convert_to_utm, axis=1)

# 输出结果
nodes

,nid,lat,lon,zone_num,zone_letter,x,y
0,0,40.583622,-3.566653,30,T,452044.820109,4.492691e+06
1,1,40.583762,-3.566087,30,T,452092.888399,4.492706e+06
2,2,40.583886,-3.565532,30,T,452139.945821,4.492720e+06
3,3,40.583988,-3.565037,30,T,452181.883934,4.492731e+06
4,4,40.584462,-3.559403,30,T,452659.024998,4.492780e+06
...,...,...,...,...,...,...,...
2913,2913,40.554195,-3.495452,30,T,458052.187436,4.489388e+06
2914,2914,40.554255,-3.495139,30,T,458078.699830,4.489395e+06
2915,2915,40.554262,-3.494986,30,T,458091.691655,4.489395e+06
2916,2916,40.553634,-3.494254,30,T,458153.217018,4.489325e+06


In [12]:
nodes['x_new'] = (nodes['x'] // 100).astype(int)#将区域划分为100米*100米方格并编号，这里是格子的数量
nodes['y_new'] = (nodes['y'] // 100).astype(int)

nodes

,nid,lat,lon,zone_num,zone_letter,x,y,x_new,y_new
0,0,40.583622,-3.566653,30,T,452044.820109,4.492691e+06,4520,44926
1,1,40.583762,-3.566087,30,T,452092.888399,4.492706e+06,4520,44927
2,2,40.583886,-3.565532,30,T,452139.945821,4.492720e+06,4521,44927
3,3,40.583988,-3.565037,30,T,452181.883934,4.492731e+06,4521,44927
4,4,40.584462,-3.559403,30,T,452659.024998,4.492780e+06,4526,44927
...,...,...,...,...,...,...,...,...,...
2913,2913,40.554195,-3.495452,30,T,458052.187436,4.489388e+06,4580,44893
2914,2914,40.554255,-3.495139,30,T,458078.699830,4.489395e+06,4580,44893
2915,2915,40.554262,-3.494986,30,T,458091.691655,4.489395e+06,4580,44893
2916,2916,40.553634,-3.494254,30,T,458153.217018,4.489325e+06,4581,44893


In [13]:
nodes['new_field'] = nodes['zone_num'].astype(str) + nodes['zone_letter'] + ' '+ nodes['x_new'].astype(str) + ' '+  nodes['y_new'].astype(str)
nodes

,nid,lat,lon,zone_num,zone_letter,x,y,x_new,y_new,new_field
0,0,40.583622,-3.566653,30,T,452044.820109,4.492691e+06,4520,44926,30T 4520 44926
1,1,40.583762,-3.566087,30,T,452092.888399,4.492706e+06,4520,44927,30T 4520 44927
2,2,40.583886,-3.565532,30,T,452139.945821,4.492720e+06,4521,44927,30T 4521 44927
3,3,40.583988,-3.565037,30,T,452181.883934,4.492731e+06,4521,44927,30T 4521 44927
4,4,40.584462,-3.559403,30,T,452659.024998,4.492780e+06,4526,44927,30T 4526 44927
...,...,...,...,...,...,...,...,...,...,...
2913,2913,40.554195,-3.495452,30,T,458052.187436,4.489388e+06,4580,44893,30T 4580 44893
2914,2914,40.554255,-3.495139,30,T,458078.699830,4.489395e+06,4580,44893,30T 4580 44893
2915,2915,40.554262,-3.494986,30,T,458091.691655,4.489395e+06,4580,44893,30T 4580 44893
2916,2916,40.553634,-3.494254,30,T,458153.217018,4.489325e+06,4581,44893,30T 4581 44893


In [14]:
print(nodes['x_new'].max()+5)
print(nodes['x_new'].min()-5)
print(nodes['y_new'].max()+5)
print(nodes['y_new'].min()-5)

4662
4515
44938
44847


In [15]:
from torchtext.vocab import build_vocab_from_iterator

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']


def yield_node_tokens():
    for x in range(nodes['x_new'].min()-5,nodes['x_new'].max()+5):
        for y in range(nodes['y_new'].min()-5,nodes['y_new'].max()+5):
            node = "30T {} {}".format(x, y)
            yield [node]

node_vocab = build_vocab_from_iterator(  yield_node_tokens(),
                                    min_freq=1,
                                    specials=special_symbols,
                                    special_first=True)
node_vocab.set_default_index(UNK_IDX)

print("NODE Vocab Size: ", len(node_vocab))
# print("Vocab Itos: ", node_vocab.get_itos() )
# print("Vocab Stoi: ", node_vocab.get_stoi() )

def yield_edge_tokens():
    for eid in edges['eid']:
        yield [str(eid)]

edge_vocab = build_vocab_from_iterator(  yield_edge_tokens(),
                                    min_freq=1,
                                    specials=special_symbols,
                                    special_first=True)
edge_vocab.set_default_index(UNK_IDX)

print("EDGE Vocab Size: ", len(edge_vocab))

NODE Vocab Size:  13381
EDGE Vocab Size:  4508


In [16]:
node_vocab(['30T 4520 44926'])

[538]

In [17]:
def coord_to_utm_str(coord):
    lat = coord[0]
    lon = coord[1]
    x, y, zone_num, zone_letter = utm.from_latlon(lat, lon)
    utm_str = "{}{} {} {}".format(zone_num, zone_letter, int(x//100), int(y//100))
    return utm_str

def coords_to_utm_strs(coords):
    return [coord_to_utm_str(coord) for coord in coords]

vocab_ids = node_vocab(coords_to_utm_strs(coords))
print(vocab_ids)


def as_strs(eids):
    return [str(eid) for eid in eids]

edge_ids = edge_vocab(as_strs(traj))
print(edge_ids)
print(traj)

print(len(vocab_ids))
print(len(traj))

print(node_vocab.lookup_tokens(vocab_ids))
print(edge_vocab.lookup_tokens(edge_ids))

[5963, 5873, 5873, 5964, 5873, 5873]
[200, 1168, 201, 203, 208, 1828]
[1174, 2045, 1175, 1177, 1181, 264]
6
6
['30T 4580 44891', '30T 4579 44892', '30T 4579 44892', '30T 4580 44892', '30T 4579 44892', '30T 4579 44892']
['1174', '2045', '1175', '1177', '1181', '264']


# 模型

In [18]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
from typing import Iterable, List

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,   #将区域划分为100米*100米方格并编号，这里是格子的数量
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [19]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [20]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(node_vocab)#将区域划分为100米*100米方格并编号，这里是格子的数量
TGT_VOCAB_SIZE = len(edge_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128

NUM_WORKERS = 8

TRAIN_SAMPLE_SIZE = 150000 # 15万
VAL_SAMPLE_SIZE = 256

NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [21]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
node_trans = sequential_transforms(coords_to_utm_strs, #Tokenization
                                   node_vocab, #Numericalization
                                   tensor_transform) # Add BOS/EOS and create tensor

edge_trans = sequential_transforms(as_strs, #Tokenization
                                   edge_vocab, #Numericalization
                                   tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(node_trans(src_sample))
        tgt_batch.append(edge_trans(tgt_sample))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [22]:
   
class NoisyTrajIterator:
    def __init__(self, n ,G):
        self.n = n  # 要迭代的次数
        self.current = 0 # 当前迭代位置
        self.G = G

    def __iter__(self):
        return self  # 返回迭代器对象（self）

    def __next__(self):
        if self.current < self.n:
            eids,coords = generate_noisy_traj(self.G)
            while len(eids) < 2 :
                 eids,coords = generate_noisy_traj(self.G)
            self.current += 1  # 定位到下一个元素
            return coords, eids
        else:
            raise StopIteration
    
    def __len__(self):
        return self.n
    
    def __getitem__(self, key):  # 每次都是随机，制定索引无效， 这里只是为了配合 DataLoader
        return self.__next__()
        
for n,e in NoisyTrajIterator(1,G):
    print(n)
    print(e)
    
print(len(NoisyTrajIterator(1120,G)))

print (NoisyTrajIterator(23,G)[12])

[[40.54247894388765, -3.5371216711742166], [40.54251460141223, -3.537535505274104], [40.5425513378431, -3.5380132038414582], [40.5425269434341, -3.538101878213174], [40.54331831420923, -3.5382586396102704], [40.543512866580976, -3.5389112126366777], [40.543773856756474, -3.5390785315185624], [40.543675108672744, -3.5395785855074053], [40.54382725780681, -3.540118654376489], [40.543612073865695, -3.5401050750083978], [40.543655704158674, -3.5404157385343287], [40.543467987710955, -3.5406224326035147], [40.54337245244819, -3.540675454682942], [40.54285800857531, -3.5414829787515005], [40.54190527531011, -3.543372876864134], [40.54153843809441, -3.544031175247652], [40.54127333264991, -3.545130695280617], [40.54048878212936, -3.5457305101645122], [40.540086327525366, -3.5462365184258764], [40.53985732740687, -3.5469711566385906], [40.54029694491578, -3.546456333219069], [40.54015219529057, -3.5462268784746516], [40.54150752765223, -3.54353798786314]]
[2589, 2589, 291, 2583, 2582, 2580, 25

In [23]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = NoisyTrajIterator(TRAIN_SAMPLE_SIZE,G)
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn, num_workers=NUM_WORKERS )

    for src, tgt in tqdm(train_dataloader):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    # return losses / len(list(train_dataloader))
    return losses / math.ceil(TRAIN_SAMPLE_SIZE / BATCH_SIZE)


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = NoisyTrajIterator(VAL_SAMPLE_SIZE,G)
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn, num_workers=NUM_WORKERS )

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
        
        # 模型保存
        checkpoint = {'model_state_dict': transformer.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict()}
        torch.save(checkpoint, "./checkpoints/checkpoint_emb_{}_head_{}_ffn_{}_enclayer_{}_declayer_{}_loss_{:.3f}.pth".format(EMB_SIZE, NHEAD, FFN_HID_DIM,NUM_ENCODER_LAYERS,NUM_DECODER_LAYERS, loss))

    #return losses / len(list(val_dataloader))
    return losses / math.ceil(VAL_SAMPLE_SIZE / BATCH_SIZE)

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


  0%|                                                                                         | 0/1172 [00:00<?, ?it/s]

In [ ]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def map_mapping(model: torch.nn.Module, src_coords: str):
    model.eval()
    src = node_trans(src_coords).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    predict = edge_vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))
    return [x for x in predict if x != '<bos>' and  x != '<eos>']

def show_map_mapping(model: torch.nn.Module, src_coords: str):
    return " ".join(map_mapping(model,src_coords)).replace("<bos>", "").replace("<eos>", "")

In [ ]:
eids,coords = generate_noisy_traj(G)
print('PREDICT -> ',show_map_mapping(transformer, coords))
print('REAL    ->  ', " ".join([str(eid) for eid in eids]))

In [ ]:
# 模型保存和加载
# loss = 0.726
# checkpoint = {'model_state_dict': transformer.state_dict(),
#               'optimizer_state_dict': optimizer.state_dict()}
# torch.save(checkpoint, "checkpoint_emb_{}_head_{}_ffn_{}_enclayer_{}_declayer_{}_loss_{:.3f}.pth".format(EMB_SIZE, NHEAD, FFN_HID_DIM,NUM_ENCODER_LAYERS,NUM_DECODER_LAYERS, loss))
# checkpoint = torch.load('model.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
eids,coords = generate_noisy_traj(G)
predict = map_mapping(transformer, coords)
print('PREDICT -> ', predict)
print('REAL    -> ', [str(eid) for eid in eids])

## 用最短路径填充间隙

In [ ]:
import networkx as nx

def gen_full_edges(edge_ids):
    full_edges = []

    last_end_node_id = None
    last_edge_id = None

    for eid in edge_ids:
        eid = int(eid)
        if last_edge_id:
            if eid == last_edge_id :
                continue
        if last_end_node_id:
            curr_start_node_id = edges.loc[eid,'start_node']
            node_path = nx.shortest_path(G, source=last_end_node_id, target=curr_start_node_id)
            if len(node_path) >1 :
                for i in range(1,len(node_path)):
                    start_node = node_path[i-1]
                    end_node = node_path[i]
                    edge_gap = edges[edges['start_node']==start_node ][ edges['end_node']==end_node ]['eid'].item()
                    full_edges += [edge_gap]
        full_edges += [eid]
        last_end_node_id = edges.loc[eid,'end_node']
        last_edge_id = eid
    return full_edges

full_predict = gen_full_edges(predict)
full_real =gen_full_edges(eids)


print(len(full_predict))
print(len(full_real))

print(json.dumps(full_predict))
print(json.dumps(full_real))

In [ ]:
# 返回最长公共子串
def longest_common_substring(A, B):
    n, m = len(A), len(B)
    L = [[0] * (m+1) for _ in range(n+1)]
    max_len, end_pos = 0, -1
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            if A[i-1] != B[j-1]:
                L[i][j] = 0
            else:
                L[i][j] = L[i-1][j-1] + 1
                if L[i][j] > max_len:
                    max_len = L[i][j]
                    end_pos = i-1
    
    begin_pos = end_pos - max_len + 1
    return A[begin_pos:end_pos+1]

common = longest_common_substring(full_predict,full_real)

print(len(common))

print(json.dumps(common))

In [ ]:
for _ in range(10):
    eids,coords = generate_noisy_traj(G)
    while len(eids) < 2 :
        eids,coords = generate_noisy_traj(self.G)
            
    predict = map_mapping(transformer, coords)
    print('PREDICT -> ', predict)
    print('REAL    -> ', [str(eid) for eid in eids])
    
    full_predict = gen_full_edges(predict)
    full_real =gen_full_edges(eids)
    print('预测路径边数 -> ',len(full_predict))
    print('真实路径边数 -> ',len(full_real))
    common = longest_common_substring(full_predict,full_real)
    print('重叠路径长度 -> ',len(common))
    print('重叠率 = 重叠路径长度 / max( 预测路径边数 , 真实路径边数 )  = {:.3f}% '.format(len(common)/max(len(full_predict),len(full_real))*100))
    print('---')

# 可视化

In [2]:
# load from file
locations=[]


file = open("data5.txt","r")
while True:
    line = file.readline()
    if not line:
        break
    tokens = line.strip().split(' ')
    locations.append([float(tokens[0]),float(tokens[1])])
file.close()

coords = locations

predict = map_mapping(transformer, coords)
full_predict = gen_full_edges(predict)

print(json.dumps(full_predict))
print('预测路径边数 -> ',len(full_predict))

FileNotFoundError: [Errno 2] No such file or directory: 'data5.txt'

In [ ]:
import folium
# define the world map
world_map = folium.Map(location=coords[0],zoom_start=13)

#画原始轨迹
for i in range(1,len(coords)):
    points = [coords[i-1], coords[i]]
    folium.PolyLine(points, color='red', weight=2).add_to(world_map)
    

#画真实
for i in range(1,len(full_real)):
    points = json.loads(edges.loc[full_real[i],'locations'])
    folium.PolyLine(points, color='green', weight=2).add_to(world_map)
    
    
#画预测
for i in range(1,len(full_predict)):
    points = json.loads(edges.loc[int(full_predict[i]),'locations'])
    folium.PolyLine(points, color='purple', weight=2).add_to(world_map)
    
world_map